In [ ]:
import os
import dask
from dask.distributed import Client
from azureml.core import Run

In [ ]:
def update_packages():
    os.system('pip install --upgrade fsspec adlfs')

In [ ]:
computes = [dask.delayed(update_packages)() for i in range(16)]
computes;

In [ ]:
run = Run.get_context()
run

from dask_cuda import LocalCUDACluster
cluster = LocalCUDACluster()

c = Client(cluster)
c.compute(computes)

In [3]:
import os
import dask
import time
import adlfs
import joblib
import fsspec
import socket
import matplotlib

import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from dask.distributed import Client
from IPython.core.display import HTML
from dask_ml.xgboost import XGBRegressor

from azureml.core import Run

%matplotlib inline

In [4]:
run = Run.get_context()
run

Experiment,Id,Type,Status,Details Page,Docs Page
dask-dask-demo,dask-dask-demo_1579762226_e5a99500,azureml.scriptrun,Running,Link to Azure Machine Learning studio,Link to Documentation


In [5]:
from dask_cuda import LocalCUDACluster

In [18]:
cluster = LocalCUDACluster()

/opt/conda/envs/rapids/lib/python3.6/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


In [19]:
help(cluster.__init__)

Help on method __init__ in module dask_cuda.local_cuda_cluster:

__init__(n_workers=None, threads_per_worker=1, processes=True, memory_limit=None, device_memory_limit=None, CUDA_VISIBLE_DEVICES=None, data=None, local_directory=None, protocol=None, enable_tcp_over_ucx=False, enable_infiniband=False, enable_nvlink=False, ucx_net_devices=None, **kwargs) method of dask_cuda.local_cuda_cluster.LocalCUDACluster instance
    Initialize self.  See help(type(self)) for accurate signature.



In [20]:
c = Client(cluster)
c

Client Scheduler: tcp://127.0.0.1:41927 Dashboard: http://127.0.0.1:38525/status,Cluster Workers: 4 Cores: 4 Memory: 473.40 GB


In [21]:
STORAGE_OPTIONS = {
    'account_name': run.experiment.workspace.datastores['gen2'].account_name,
    'account_key' : run.experiment.workspace.datastores['gen2'].account_key
}

In [22]:
protocol   = 'abfs'
filesystem = 'datasets'

In [23]:
data_path = f'{protocol}://{filesystem}'
data_path

'abfs://datasets'

In [24]:
#data_path = f'{run.get_metrics()["datastore"]}'
#data_path

In [25]:
fs = fsspec.filesystem(protocol, **STORAGE_OPTIONS, container_name=filesystem)

In [26]:
files = []
for file in fs.glob('noaa/isd/year=*/month=*'): # see https://github.com/dask/adlfs/issues/34
    files += fs.ls(f'{file}/')
files = [f'{data_path}/{file}' for file in files if '2019' not in file] # issue with 2019 data - fix wip 
files[-5:]

['abfs://datasets/noaa/isd/year=2018/month=5/part-00066-tid-9138739344806125380-ef942066-1d58-49f9-8ecb-3329cbe6e57e-383761.c000.snappy.parquet',
 'abfs://datasets/noaa/isd/year=2018/month=6/part-00049-tid-9138739344806125380-ef942066-1d58-49f9-8ecb-3329cbe6e57e-383744.c000.snappy.parquet',
 'abfs://datasets/noaa/isd/year=2018/month=7/part-00107-tid-9138739344806125380-ef942066-1d58-49f9-8ecb-3329cbe6e57e-383859.c000.snappy.parquet',
 'abfs://datasets/noaa/isd/year=2018/month=8/part-00103-tid-9138739344806125380-ef942066-1d58-49f9-8ecb-3329cbe6e57e-383807.c000.snappy.parquet',
 'abfs://datasets/noaa/isd/year=2018/month=9/part-00089-tid-9138739344806125380-ef942066-1d58-49f9-8ecb-3329cbe6e57e-383784.c000.snappy.parquet']

In [27]:
df = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute()
%time df.head()

CPU times: user 304 ms, sys: 62.7 ms, total: 366 ms
Wall time: 5.57 s


,usaf,wban,datetime,latitude,longitude,elevation,windAngle,windSpeed,temperature,seaLvlPressure,...,pastWeatherIndicator,precipTime,precipDepth,snowDepth,stationName,countryOrRegion,p_k,year,day,version
0,724755,93129,2008-01-01 00:53:00,37.702,-113.097,1712.0,350.0,3.6,-8.3,1036.2,...,NaN,1.0,0.0,NaN,CEDAR CITY REGIONAL ARPT,US,724755-93129,2008,1,1.0
1,724755,93129,2008-01-01 01:53:00,37.702,-113.097,1712.0,340.0,4.1,-11.1,1037.3,...,NaN,1.0,0.0,NaN,CEDAR CITY REGIONAL ARPT,US,724755-93129,2008,1,1.0
2,724755,93129,2008-01-01 02:53:00,37.702,-113.097,1712.0,NaN,2.1,-11.7,1039.7,...,NaN,1.0,0.0,NaN,CEDAR CITY REGIONAL ARPT,US,724755-93129,2008,1,1.0
3,724755,93129,2008-01-01 03:53:00,37.702,-113.097,1712.0,70.0,1.5,-11.1,1040.3,...,NaN,1.0,0.0,NaN,CEDAR CITY REGIONAL ARPT,US,724755-93129,2008,1,1.0
4,724755,93129,2008-01-01 04:53:00,37.702,-113.097,1712.0,NaN,1.5,-13.9,1040.4,...,NaN,1.0,0.0,NaN,CEDAR CITY REGIONAL ARPT,US,724755-93129,2008,1,1.0


In [32]:
import dask_cudf

In [33]:
df = dask_cudf.from_dask_dataframe(df)

In [ ]:
#df = df.set_index(dd.to_datetime(df.datetime).dt.floor('d'), sorted=False)
#df = df.persist() 
%time len(df)

In [ ]:
%time len(df)

In [ ]:
%time df.describe().compute()

In [ ]:
%time bites = df.memory_usage(index=True, deep=True).sum().compute()
print(f'Dataframe is: {round(bites/1e9, 2)}GB')

In [ ]:
%time means = df.groupby(df.index).mean().compute()
means.head()

In [ ]:
for col in list(means.columns):
    fig = plt.figure(figsize=(16, 8))
    #plt.style.use('dark_background')
    means[col].plot(color='b')
    plt.title('Average of {}'.format(col))
    plt.xlim([datetime(2008, 1, 1), datetime(2018, 12, 31)])
    plt.grid()
    
    # optionally, log the image to the run
    run.log_image(f'mean_{col}', plot=plt)

You can see the images logged to the run in the studio.

In [ ]:
run

## Prepare data

In [ ]:
## insert any Pandas-like Dask data prep code 
df['temperature'] = df['temperature']*(9/5)+32 # 'Merica

## Write data

**Important:** you will have not access to write to this account, get your own storage account! 

See the [API documentation for dask.dataframe.to_parquet](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.to_parquet) for additional settings. 

In [ ]:
df = df.repartition(npartitions=df.npartitions*10)
%time dask.delayed(df.to_parquet)(f'abfs://outputs/noaa/isd_out.parquet', compression='lz4', storage_options=STORAGE_OPTIONS).compute()

![Write gif](media/write.gif)

## Train XGBoost model

In [ ]:
df = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute()

In [ ]:
files = []
for file in fs.glob('noaa/isd/year=*/month=*'): # see https://github.com/dask/adlfs/issues/34
    files += fs.ls(f'{file}/')
files = [f'{protocol}://{container}/{file}' for file in files if '2019' in file] 

df2019 = dask.delayed(dd.read_parquet)(files, engine='pyarrow', storage_options=STORAGE_OPTIONS).compute() 

In [ ]:
# begin data prep
df = df.fillna(0) 
df2019 = df2019.fillna(0) 

In [ ]:
df['month'] = df['datetime'].dt.month
df2019['month'] = df2019['datetime'].dt.month

In [ ]:
cols = list(df.columns)
cols = [col for col in cols if df.dtypes[col] != 'object' and col not in ['version', 'datetime']]
cols

In [ ]:
X = df[[col for col in cols if col not in ['temperature']]].persist()
y = df.temperature.persist()
# end data prep - persist intelligently per https://docs.dask.org/en/latest/best-practices.html

In [ ]:
xgb = XGBRegressor(n_estimators=16)
%time xgb.fit(X, y)

In [ ]:
%time y_pred = xgb.predict(X).compute()

In [ ]:
rmse = (((y.to_dask_array().compute()-y_pred)**2).mean())**.5 # runs locally, distribute (?)
print(f'Training RMSE: {round(rmse, 3)}')

## Evaluate model

In [ ]:
X_test = df2019[[col for col in cols if col not in ['temperature']]].persist()
y_test = df2019.temperature.persist()

In [ ]:
%time y_pred = xgb.predict(X_test).compute()

In [ ]:
rmse = (((y_test.to_dask_array().compute()-y_pred)**2).mean())**.5 # runs locally, distribute (?)
print(f'Test RMSE: {round(rmse, 3)}')

## Register model

In [ ]:
model_path = 'xgboost_noaa_isd.joblib.dat'
joblib.dump(xgb, model_path)
xgb = joblib.load(model_path)

In [ ]:
model = Model.register(ws, model_path, 'xgboost-noaa-isd', 
                       description='Dask XGBoost NOAA ISD temperature predictor',
                       model_framework='XGBoost')

## End the run

Cluster will return to 0 nodes.

In [ ]:
c.close()
run.cancel()

In [ ]:
t_end = time.time()
print(f'Total run time: {round((t_end-t_start)/60, 2)} minutes')